In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import math

np.random.seed(42)

def gaussian_2d(x, y, amplitude, x_mean, y_mean, x_stddev, y_stddev, theta=0):
    """
    2D Gaussian function.
    """
    a = np.cos(theta)**2 / (2 * x_stddev**2) + np.sin(theta)**2 / (2 * y_stddev**2)
    b = -np.sin(2 * theta) / (4 * x_stddev**2) + np.sin(2 * theta) / (4 * y_stddev**2)
    c = np.sin(theta)**2 / (2 * x_stddev**2) + np.cos(theta)**2 / (2 * y_stddev**2)
    exponent = a * (x - x_mean)**2 + 2 * b * (x - x_mean) * (y - y_mean) + c * (y - y_mean)**2
    return amplitude * np.exp(-exponent)

x_lin = np.linspace(-1, 1, 400)
y_lin = np.linspace(-1, 1, 400)
X, Y = np.meshgrid(x_lin, y_lin)

Z1 = gaussian_2d(X, Y, 1.0, -.75, .5, .50, 2, math.pi/4)
Z2 = gaussian_2d(X, Y, 0.5, -.2, .5, .4, 2, -math.pi/4) * 2
Z3 = gaussian_2d(X, Y, 1.0, .75, 0.15, .25, 3, math.pi/3) * -.5
Z4 = gaussian_2d(X, Y, 1.0, 1.05, 0.2, .25, 3, math.pi/3) * -.5

r = np.random.randn(*Z1.shape) * 2
r = gaussian_filter(r, 12)
Z = Z1 + Z2 + Z3 + Z4 + r

g0, g1 = np.gradient(Z)
g = (g1**2 + g0**2) ** 0.5

fig, ax = plt.subplots(1, 4)
ax[0].imshow(Z1, cmap='viridis')
ax[1].imshow(Z2, cmap='viridis')
ax[2].imshow(Z3, cmap='viridis')
ax[3].imshow(Z4, cmap='viridis')

plt.show()
plt.imshow(Z, cmap='terrain')
plt.colorbar()
plt.show()

#plt.imshow(g, cmap='viridis')
#plt.colorbar()

from tests.datagen import save_tensor_to_csv
import torch
save_tensor_to_csv(torch.Tensor(Z), "static_data/adam_v.csv")
save_tensor_to_csv(torch.Tensor(g0), "/home/ishwark/transformer-c/adam_g0.csv")
save_tensor_to_csv(torch.Tensor(g1), "/home/ishwark/transformer-c/adam_g1.csv")

In [ ]:
# plot from C++ gen files
import numpy as np
import pandas as pd
plt.rcParams['figure.dpi'] = 200
df = pd.read_csv('xy.csv')
xs = np.array(df['x0'])
ys = np.array(df['x1'])
#gx = np.array(df['gx'])
#gy = np.array(df['gy'])
plt.imshow(Z, cmap='terrain')
plt.colorbar()
plt.plot(ys, xs, 'b.',  markersize=.75)
min_at = np.unravel_index(np.argmin(Z), Z.shape)
plt.plot(ys[0], xs[0], 'gx', markersize=2)
plt.plot(ys[-1], xs[-1], 'rx', markersize=2)
print("min: ", min_at, Z[min_at], g[min_at])
plt.plot(min_at[1], min_at[0], 'ro')

In [ ]:
import torch
from scipy.interpolate import RegularGridInterpolator
x, y = 120, 300
p = torch.nn.Parameter(torch.Tensor([x/Z.shape[0], y/Z.shape[1]]), requires_grad=True)

optim = torch.optim.Adam([p], lr=3e-2, betas=[0.9, 0.999])
p.sum().backward()  # to add grad attribute
h, w = Z.shape
interp_gx = RegularGridInterpolator((range(h), range(w)), g1, method='linear')
interp_gy = RegularGridInterpolator((range(h), range(w)), g0, method='linear')
interp_vz = RegularGridInterpolator((range(h), range(w)), Z, method='linear')
xs, ys = [], []
for b in range(301):
    x, y = p[0].item() * Z.shape[0], p[1].item() * Z.shape[1]
    if x < 0 or x >= Z.shape[0]-1 or y < 0 or y >= Z.shape[1] - 1:
        break
    gx_ = interp_gx([y, x]).item()
    gy_ = interp_gy([y, x]).item()
    p.grad = torch.FloatTensor([gx_, gy_])
    if(gx_ < 1e-7 and gy_ < 1e-7):
        break
    optim.step()
    if b % 20 == 0:
        px, py = p.data[0].item(), p.data[1].item()
        gy_, gx_ = p.grad[0].item(), p.grad[1].item()
        if px >=0 and px < Z.shape[0]-1 and py >= 0 and py < Z.shape[1]-1:
            val = interp_vz([y, x]).item()
            print(f'{b:3d} pixel: [{x:10.8f}, {y:10.8f}] metric: [{px:10.8f}, {py:10.8f}] grad: [{gx_:10.8f}, {gy_:10.8f}] value: {val:10.8f} ')
    xs.append(x)
    ys.append(y)
plt.imshow(Z, cmap='terrain')
plt.colorbar()
plt.plot(xs, ys, 'b.', markersize=.75)
plt.plot(xs[0], ys[0], 'gx', markersize=2)
plt.plot(xs[-1], ys[-1], 'rx', markersize=2)
min_at = np.unravel_index(np.argmin(Z), Z.shape)
print("min: ", min_at, Z[min_at], g[min_at])
plt.plot(min_at[1], min_at[0], 'ko')
plt.show()

plt.imshow(g, cmap='viridis')
plt.colorbar()
plt.plot(xs, ys, 'rx', markersize=.75)